In [1]:
import pandas as pd
#import numpy as np
from pandas.core.frame import DataFrame

import pickle

In [ ]:
# read the dataset

In [2]:
f1 = open('customer_service_reps','rb')
csr = pickle.load(f1)
#csr.head()

In [3]:
f2 = open('engagement','rb')
engagement = pickle.load(f2)
#engagement.head()

In [4]:
f3 = open('subscribers','rb')
subscribers = pickle.load(f3)
#subscribers.head()

In [10]:
#subscribers.columns.values.tolist()
#csr.columns.values.tolist()
#engagement.columns.values.tolist()
#print(subscribers['account_creation_date'].min())
# max: 2020-03-27 23:59:06
# min: 2019-06-30 00:00:19

2019-06-30 00:00:19


In [5]:
advertising_spend = pd.read_excel('advertising_spend_data.xlsx')
advertising_spend.columns = advertising_spend.iloc[1]
advertising_spend = advertising_spend.drop([0,1])
#advertising_spend.head()

In [6]:
asdf = subscribers[['attribution_technical','attribution_survey']]
asdf

,attribution_technical,attribution_survey
162502,facebook,facebook
380374,facebook,facebook
496617,organic,facebook
356799,search,tv
681701,discovery,youtube
...,...,...
509326,facebook,facebook_organic
229587,google_organic,referral
281340,facebook,facebook
87198,facebook,facebook


In [8]:
asdf['attribution_survey'].value_counts()

facebook            119126
tv                   39904
referral             20882
search                8492
pinterest             7856
other                 6496
public_radio          6219
social_organic        3869
youtube               3108
podcast               2995
affiliate             1962
ott                   1443
sirius xm             1122
facebook_organic       765
direct_mail            408
amfm_radio             337
Name: attribution_survey, dtype: int64

# Attribution and Allocation

##### NOTES

In [ ]:
???????????????????
# spending 和 coming user可以分月，需要做分月的吗

In [ ]:
# AA_2 is dataframe for number of new coming users of technical 
# AA_3 is dataframe for number of new coming users of survey
# AA_4 is dataframe that contains crdites by channel
# AA_spend is dataframe that contains spending by channel
# AA_final contains average CAC
# revenue_1 drop rows that do not belong to channel list
# revenue_2 tech revenue by channel
# revenue_3 surv revenue by channel

In [ ]:
# use both techinal and survey channel, and credits 50% to each
# use discount-fee to calculate revenue

### (1)CAC

#####  1a. Calculate credits——position based model

In [ ]:
# attribution_technical: entrance to sign-up form captured by product (that is last touch point)
# attribution_survey: customer reported "how did you hear about us", self-reported upon signup (arbitrarily regard it as the first touch point)
# since both touch point is important, so I use position based model, give 50% and 50% to each channel

In [13]:
# create a new dataframe which named AA = Attribution and Allocation to calculate CAC and CLV
AA = subscribers[['attribution_technical','attribution_survey']]
#AA
# 227628 rows, but contains several channels that we do not have budget information, drop them out

,attribution_technical,attribution_survey
162502,facebook,facebook
380374,facebook,facebook
496617,organic,facebook
356799,search,tv
681701,discovery,youtube
...,...,...
509326,facebook,facebook_organic
229587,google_organic,referral
281340,facebook,facebook
87198,facebook,facebook


In [30]:
#AA['attribution_technical'].value_counts()

In [31]:
#AA['attribution_survey'].value_counts()

In [31]:
#advertising_spend.columns.values.tolist()

['date',
 'facebook',
 'email',
 'search',
 'brand sem intent google',
 'affiliate',
 'email_blast',
 'pinterest',
 'referral']

In [14]:
# drop the row that do not belong to following 5 channels
channels = ['facebook','email','search','brand sem intent google','affiliate','email_blast','pinterest','referral']
AA_1 = AA.loc[AA['attribution_technical'].isin(channels)]
AA_1 = AA_1.loc[AA['attribution_survey'].isin(channels)]
AA_1 
#134346  rows

,attribution_technical,attribution_survey
162502,facebook,facebook
380374,facebook,facebook
315863,email,referral
254782,facebook,facebook
681969,search,facebook
...,...,...
230734,facebook,facebook
577745,affiliate,facebook
603751,email,pinterest
281340,facebook,facebook


In [35]:
#AA_1['attribution_technical'].value_counts() # facebook     69222
#AA_1['attribution_survey'].value_counts()# facebook    102968

In [15]:
# calculate attribution_technical credits
AA_2=AA_1.groupby('attribution_technical',as_index = False).count()
AA_2.rename(columns={'attribution_technical':'channel'},inplace = True)
AA_2.rename(columns={'attribution_survey':'attribution_technical'},inplace = True)
AA_2['tech_coeff'] = 0.5
# calculate credits
AA_2['credit'] = AA_2['attribution_technical']* AA_2['tech_coeff']
AA_2

,channel,attribution_technical,tech_coeff,credit
0,affiliate,6345,0.5,3172.5
1,brand sem intent google,9778,0.5,4889.0
2,email,18266,0.5,9133.0
3,email_blast,5433,0.5,2716.5
4,facebook,69222,0.5,34611.0
5,pinterest,5076,0.5,2538.0
6,referral,4770,0.5,2385.0
7,search,15456,0.5,7728.0


In [16]:
AA_3=AA_1.groupby('attribution_survey',as_index = False).count()
AA_3.rename(columns={'attribution_survey':'channel'},inplace = True)
AA_3.rename(columns={'attribution_technical':'attribution_survey'},inplace = True)
AA_3['surv_coeff'] = 0.5
# calculate credits
AA_3['credit'] = AA_3['attribution_survey']* AA_3['surv_coeff']
AA_3

,channel,attribution_survey,surv_coeff,credit
0,affiliate,1615,0.5,807.5
1,facebook,102968,0.5,51484.0
2,pinterest,6800,0.5,3400.0
3,referral,15903,0.5,7951.5
4,search,7060,0.5,3530.0


In [17]:
AA_4 = pd.merge(AA_2,AA_3,how = 'outer',on ='channel' )
#replace NaN to 0
AA_4 = AA_4.fillna(0)
# calculate total credits
AA_4['total_credits'] = AA_4['credit_x']+ AA_4['credit_y']
AA_4

,channel,attribution_technical,tech_coeff,credit_x,attribution_survey,surv_coeff,credit_y,total_credits
0,affiliate,6345,0.5,3172.5,1615.0,0.5,807.5,3980.0
1,brand sem intent google,9778,0.5,4889.0,0.0,0.0,0.0,4889.0
2,email,18266,0.5,9133.0,0.0,0.0,0.0,9133.0
3,email_blast,5433,0.5,2716.5,0.0,0.0,0.0,2716.5
4,facebook,69222,0.5,34611.0,102968.0,0.5,51484.0,86095.0
5,pinterest,5076,0.5,2538.0,6800.0,0.5,3400.0,5938.0
6,referral,4770,0.5,2385.0,15903.0,0.5,7951.5,10336.5
7,search,15456,0.5,7728.0,7060.0,0.5,3530.0,11258.0


##### 1b.calculate channel spending

In [18]:
advertising_spend.head()

1,date,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
2,2019-06-30 00:00:00,84800,75000,17300,21400,11900,10500,5900,6100
3,2019-07-31 00:00:00,60000,75600,17300,21300,12000,29800,6000,6600
4,2019-08-31 00:00:00,52300,76100,18100,21300,16300,65900,6200,6800
5,2019-09-30 00:00:00,49000,75800,18900,21300,16400,73700,6000,7400
6,2019-10-31 00:00:00,51300,76000,21000,21400,12200,32300,6700,7300


In [19]:
a = dict()
for i in list(advertising_spend.columns)[1:]:
    #print(sum(advertising_spend[i]))
    a[i] = sum(advertising_spend[i])
#a
AA_spend = pd.DataFrame(a,index = ['spending']).T
AA_spend['channel'] = AA_spend.index
AA_spend

,spending,channel
facebook,552700,facebook
email,760600,email
search,232500,search
brand sem intent google,216100,brand sem intent google
affiliate,139500,affiliate
email_blast,226400,email_blast
pinterest,63300,pinterest
referral,73400,referral


##### 1c.Calculate CAC

In [20]:
AA_4

,channel,attribution_technical,tech_coeff,credit_x,attribution_survey,surv_coeff,credit_y,total_credits
0,affiliate,6345,0.5,3172.5,1615.0,0.5,807.5,3980.0
1,brand sem intent google,9778,0.5,4889.0,0.0,0.0,0.0,4889.0
2,email,18266,0.5,9133.0,0.0,0.0,0.0,9133.0
3,email_blast,5433,0.5,2716.5,0.0,0.0,0.0,2716.5
4,facebook,69222,0.5,34611.0,102968.0,0.5,51484.0,86095.0
5,pinterest,5076,0.5,2538.0,6800.0,0.5,3400.0,5938.0
6,referral,4770,0.5,2385.0,15903.0,0.5,7951.5,10336.5
7,search,15456,0.5,7728.0,7060.0,0.5,3530.0,11258.0


In [21]:
AA_final =pd.merge(AA_4,AA_spend,how = 'outer',on ='channel' )
AA_final = AA_final.loc[:,['channel','total_credits','spending']]
AA_final['CAC']= AA_final['spending']/AA_final['total_credits']
AA_final

,channel,total_credits,spending,CAC
0,affiliate,3980.0,139500,35.050251
1,brand sem intent google,4889.0,216100,44.201268
2,email,9133.0,760600,83.280412
3,email_blast,2716.5,226400,83.342536
4,facebook,86095.0,552700,6.419653
5,pinterest,5938.0,63300,10.660155
6,referral,10336.5,73400,7.101050
7,search,11258.0,232500,20.651981


### (2) CLV

##### 2.1 calculate revenue by channel

In [ ]:
# since discount price is the actual price the company gain, so we use discount_price instead of monthly_price

In [22]:
revenue = subscribers.loc[:,['attribution_technical','attribution_survey','discount_price']]

channels = ['facebook','email','search','brand sem intent google','affiliate','email_blast','pinterest','referral']
revenue_1 = revenue.loc[revenue['attribution_technical'].isin(channels)]
revenue_1 = revenue_1.loc[revenue['attribution_survey'].isin(channels)]
#revenue_1 ##134346 rows

In [167]:
#subscribers['discount_price'].value_counts()

In [23]:
revenue_1['tech_revenue'] = 0.5 * revenue_1['discount_price']
revenue_1['surv_revenue'] = 0.5 * revenue_1['discount_price']
revenue_1

,attribution_technical,attribution_survey,discount_price,tech_revenue,surv_revenue
162502,facebook,facebook,4.5141,2.25705,2.25705
380374,facebook,facebook,4.5141,2.25705,2.25705
315863,email,referral,4.5141,2.25705,2.25705
254782,facebook,facebook,4.5141,2.25705,2.25705
681969,search,facebook,4.5141,2.25705,2.25705
...,...,...,...,...,...
230734,facebook,facebook,4.5141,2.25705,2.25705
577745,affiliate,facebook,4.5141,2.25705,2.25705
603751,email,pinterest,4.5141,2.25705,2.25705
281340,facebook,facebook,4.5141,2.25705,2.25705


In [24]:
revenue_2=revenue_1.groupby('attribution_technical',as_index = False)['tech_revenue'].sum()
revenue_2.rename(columns={'attribution_technical':'channel'},inplace = True)
revenue_2

,channel,tech_revenue
0,affiliate,14321.78965
1,brand sem intent google,22073.03150
2,email,41226.76150
3,email_blast,12261.89205
4,facebook,156282.36250
5,pinterest,11460.63930
6,referral,10765.98170
7,search,34894.91050


In [25]:
revenue_3=revenue_1.groupby('attribution_survey',as_index = False)['surv_revenue'].sum()
revenue_3.rename(columns={'attribution_survey':'channel'},inplace = True)
revenue_3

,channel,surv_revenue
0,affiliate,3645.172450
1,facebook,232454.937399
2,pinterest,15351.793500
3,referral,35897.756350
4,search,15937.709000


In [26]:
revenue_4 = pd.merge(revenue_2,revenue_3,how = 'outer',on ='channel' )
#replace NaN to 0
revenue_4 = revenue_4.fillna(0)
# calculate total credits
revenue_4['total_revenue'] = revenue_4['tech_revenue']+ revenue_4['surv_revenue']
revenue_4

,channel,tech_revenue,surv_revenue,total_revenue
0,affiliate,14321.78965,3645.172450,17966.962100
1,brand sem intent google,22073.03150,0.000000,22073.031500
2,email,41226.76150,0.000000,41226.761500
3,email_blast,12261.89205,0.000000,12261.892050
4,facebook,156282.36250,232454.937399,388737.299899
5,pinterest,11460.63930,15351.793500,26812.432800
6,referral,10765.98170,35897.756350,46663.738050
7,search,34894.91050,15937.709000,50832.619500


##### 2.2 calculate clv by channel

In [27]:
revenue_5 = pd.merge(revenue_4,AA_spend,how = 'outer',on ='channel' )
revenue_5 = revenue_5.loc[:,['channel','total_revenue','spending']]
revenue_5['total_credits'] = AA_final['total_credits']
revenue_5

,channel,total_revenue,spending,total_credits
0,affiliate,17966.962100,139500,3980.0
1,brand sem intent google,22073.031500,216100,4889.0
2,email,41226.761500,760600,9133.0
3,email_blast,12261.892050,226400,2716.5
4,facebook,388737.299899,552700,86095.0
5,pinterest,26812.432800,63300,5938.0
6,referral,46663.738050,73400,10336.5
7,search,50832.619500,232500,11258.0


In [28]:
revenue_5['CLV'] = (revenue_5['total_revenue'] - revenue_5['spending'])/revenue_5['total_credits']
revenue_5

,channel,total_revenue,spending,total_credits,CLV
0,affiliate,17966.962100,139500,3980.0,-30.535939
1,brand sem intent google,22073.031500,216100,4889.0,-39.686433
2,email,41226.761500,760600,9133.0,-78.766368
3,email_blast,12261.892050,226400,2716.5,-78.828680
4,facebook,388737.299899,552700,86095.0,-1.904439
5,pinterest,26812.432800,63300,5938.0,-6.144757
6,referral,46663.738050,73400,10336.5,-2.586588
7,search,50832.619500,232500,11258.0,-16.136737


In [ ]:
##### 2.3 compare CAC and CLV

In [29]:
compare = AA_final.loc[:,['channel','CAC']]
compare['CLV'] = revenue_5['CLV']
compare['CLV/CAC'] = compare['CLV']/compare['CAC']
compare

,channel,CAC,CLV,CLV/CAC
0,affiliate,35.050251,-30.535939,-0.871205
1,brand sem intent google,44.201268,-39.686433,-0.897857
2,email,83.280412,-78.766368,-0.945797
3,email_blast,83.342536,-78.828680,-0.945840
4,facebook,6.419653,-1.904439,-0.296658
5,pinterest,10.660155,-6.144757,-0.576423
6,referral,7.101050,-2.586588,-0.364254
7,search,20.651981,-16.136737,-0.781365
